In [1]:
import pandas as pd

It is easy to generate papers summary for any area. See more at README




In [2]:
from html.parser import HTMLParser

paper_session = []
class MyHTMLParser(HTMLParser):
    
    def handle_starttag(self, tag, attrs):
        self.data = attrs

    def handle_endtag(self, tag):
        pass

    def handle_data(self, data):
        if len(data) > 2:
            #print("Encountered some data  :", data)
            paper_session.append([self.data, data])
        

parser = MyHTMLParser()


file = open('./rlpapers.txt')
lines = file.read()

parser.feed(lines)

In [3]:
full_list = []
def session_processing(s):
    return [s[1],s[0][0][1]]
for s in paper_session:
    full_list.append(session_processing(s))

In [4]:
print("There are {} papers in your selection.".format(len(full_list)))

There are 159 papers in your selection.


In [5]:
import requests

In [6]:
request1 = requests.get('https://ras.papercept.net/conferences/conferences/ICRA21/program/ICRA21_ContentListWeb_1.html')
request2 = requests.get('https://ras.papercept.net/conferences/conferences/ICRA21/program/ICRA21_ContentListWeb_2.html')
request3 = requests.get('https://ras.papercept.net/conferences/conferences/ICRA21/program/ICRA21_ContentListWeb_3.html')

In [7]:
data1 = request1.content
data2 = request2.content
data3 = request3.content

In [8]:
[data1, data2, data3];

In [9]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(data1, 'html.parser')

In [10]:
soup1, soup2, soup3 = [BeautifulSoup(data1, 'html.parser'), 
                       BeautifulSoup(data2, 'html.parser'),
                       BeautifulSoup(data3, 'html.parser')]

In [11]:
soups = [soup1, soup2, soup3]

In [12]:
def summary_data(paper_info):
    web_page,paper_id = paper_info[-1].rsplit('#')
    soup = soups[int(web_page.split('.')[0][-1])-1]
    a = soup.find('a', {"name":paper_id})
    title = a.find_next('a', {"title":"Click to show or hide the keywords and abstract (text summary)"}).string
    abstract = a.find_next('div').text.rsplit('Abstract: ')[-1]
    author_list = []
    for name in a.find_all_next('a'):
        try:
            name["name"] != None
            break
        except:
            if name["title"] == "Click to go to the Author Index":
                author_list.append(name.text)
    aff_list = []
    td_counter = 0
    for name in a.find_all_next("td", {"class":"r"}):
        if "Add to" in name.text:
            if td_counter == 1:
                break
            td_counter += 1
        else:
            aff_list.append(name.text)
            
    return title, abstract, author_list, aff_list

In [13]:
rl_icra = []

In [14]:
for s in full_list:
    rl_icra.append(summary_data(s))

In [15]:
df = pd.DataFrame(data=rl_icra)

In [16]:
df.columns  = ["Title", "Abstract","Authors", "Affiliations"]

In [17]:
df.to_csv("ICRA_long.csv")

In [18]:
au_af = []

In [19]:
for i, row in df.iterrows():
    j = 0 
    for r in row["Authors"]:
        au_af.append(r + "_" +row["Affiliations"][j])
        j += 1

In [20]:
from collections import Counter

In [21]:
ct = Counter(au_af)

In [22]:
for i, row in df.iterrows():
    j = 0
    max_p = 0
    for r in row["Authors"]:
        a = r + "_" +row["Affiliations"][j]
        if ct[a] > max_p:
            max_p = ct[a]
        au_af.append(a)
        j += 1
    df.at[i,"max_p"] = max_p
        

In [23]:
df

,Title,Abstract,Authors,Affiliations,max_p
0,Towards Efficient Multiview Object Detection w...,Active vision is a desirable perceptual featur...,"[Xu, Qianli, Fang, Fen, Gauthier, Nicolas, Lia...","[Institute for Infocomm Research, I2R, Institu...",1.0
1,Deep Reinforcement Learning Framework for Unde...,Soft robotics is an emerging technology with e...,"[Li, Guanda, Shintake, Jun, Hayashibe, Mitsuhiro]","[Tohoku University, University of Electro-Comm...",2.0
2,Robotic Imitation of Human Assembly Skills Usi...,Robotic assembly tasks involve complex and low...,"[Wang, Yan, Beltran-Hernandez, Cristian Camilo...","[Osaka University, Osaka University, Osaka Uni...",1.0
3,Context-Aware Safe Reinforcement Learning for ...,Safety is a critical concern when deploying re...,"[Chen, Baiming, Liu, Zuxin, Zhu, Jiacheng, Xu,...","[Tsinghua University, Carnegie Mellon Universi...",2.0
4,Quantification of Joint Redundancy Considering...,The robotic joint redundancy for executing a t...,"[Chai, Jiazheng, Hayashibe, Mitsuhiro]","[Tohoku University, Tohoku University]",2.0
...,...,...,...,...,...
154,Multi-Target Coverage with Connectivity Mainte...,This paper considers a multi-target coverage p...,"[Wu, Shiguang, Pu, Zhiqiang, Liu, Zhen, Qiu, T...","[Chinese Academy of Sciences Beijing, China, U...",1.0
155,Remote-Center-Of-Motion Recommendation Toward ...,Brain needle intervention is a typical diagnos...,"[Gao, Huxin, Xiao, Xiao, Qiu, Liang, Meng, Max...","[National University of Singapore, Southern Un...",1.0
156,A General Approach for the Automation of Hydra...,This article presents a general approach to de...,"[Egli, Pascal Arturo, Hutter, Marco]","[RSL, ETHZ, ETH Zurich]",1.0
157,Relational Navigation Learning in Continuous A...,"In this paper, a novel navigation learning met...","[Zhang, Xueyou, Xi, Wei, Guo, Xian, Fang, Yong...","[Nankai University, Nankai University, Nankai ...",1.0


In [24]:
df = df.sort_values(by=["max_p"],ascending=False)

In [25]:
df

,Title,Abstract,Authors,Affiliations,max_p
137,Reset-Free Reinforcement Learning Via Multi-Ta...,Reinforcement Learning (RL) algorithms can in ...,"[Gupta, Abhishek, Yu, Justin, Zhao, Zihao, Kum...","[UC Berkeley, UC Berkeley, UC Berkeley, Univ. ...",9.0
120,DisCo RL: Distribution-Conditioned Reinforceme...,Can we use reinforcement learning to instead l...,"[Nasiriany, Soroush, Pong, Vitchyr, Nair, Ashv...","[UC Berkeley, UC Berkeley, UC Berkeley, UC Ber...",9.0
33,What Can I Do Here? Learning New Skills by Ima...,A generalist robot equipped with learned skill...,"[Khazatsky, Alexander, Nair, Ashvin, Jing, Dan...","[UC Berkeley, UC Berkeley, University of Calif...",9.0
134,Reinforcement Learning for Robust Parameterize...,Developing robust walking controllers for bipe...,"[Li, Zhongyu, Cheng, Xuxin, Peng, Xue Bin, Abb...","[University of California, Berkeley, Universit...",9.0
26,Model-Based Meta-Reinforcement Learning for Fl...,Transporting suspended payloads is challenging...,"[Belkhale, Suneel, Kahn, Gregory, McAllister, ...","[Stanford University, University of California...",9.0
...,...,...,...,...,...
52,A Peg-In-Hole Task Strategy for Holes in Concrete,A method that enables an industrial robot to a...,"[Yasutomi, André Yuji, Mori, Hiroki, Ogata, Te...","[Hitachi Ltd, Waseda University, Waseda Univer...",1.0
51,Learning from Demonstration without Demonstrat...,State-of-the-art reinforcement learning (RL) a...,"[Blau, Tom, Morere, Philippe, Francis, Gilad]","[University of Sydney, University of Sydney, T...",1.0
50,Dreaming: Model-Based Reinforcement Learning b...,"In the present paper, we propose a decoder-fre...","[Okada, Masashi, Taniguchi, Tadahiro]","[Panasonic Corporation, Ritsumeikan University]",1.0
49,Sample Efficient Reinforcement Learning Via Mo...,Model-based deep reinforcement learning has ac...,"[Yao, Yao, Xiao, Li, An, Zhicheng, Zhang, Wanp...","[Tsinghua-Berkeley Shenzhen Institute, Tsinghu...",1.0


In [26]:
df.to_csv("ICRA_long_sorted.csv")

In [27]:
short_info = []

In [28]:
for i, row in df.iterrows():
    short_info.append([row["Title"], row["Authors"][0], row["Affiliations"][0]])

In [29]:
pd.DataFrame(short_info).to_csv("ICRA_short.csv")